# using Formulation 

In [2]:
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv('data/play_tennis.csv')
data = data.drop('day',axis=1)
data

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


### find prediction for *play* for inputs 

* outlook == sunny
* temp == cold
* humidity == low
* wind == weak

In [8]:
data['play'].value_counts()

Yes    9
No     5
Name: play, dtype: int64

* P(Yes | (overcast,cold,low,weak)) = P(Yes) * P(sunny|Yes) * P(cold|Yes) * P(low|Yes) * P(weak|Yes)
* P(No | (overcast,cold,low,weak)) = P(No) * P(sunny|No) * P(cold|No) * P(low|No) * P(weak|No)

In [9]:
# P(Yes)
pyes = 9/14
# P(No)
pno = 5/14

* Creating lookup table for all probabilities of each columns

In [13]:
#outlook
pd.crosstab(data['outlook'],data['play'])

play,No,Yes
outlook,,
Overcast,0,4
Rain,2,3
Sunny,3,2


In [14]:
p_oc_yes = 4/9
p_rain_yes = 3/9
p_sun_yes = 2/9

p_oc_no = 0/5
p_rain_no = 2/5
p_sun_no = 3/5

In [15]:
#temp
pd.crosstab(data['temp'],data['play'])

play,No,Yes
temp,,
Cool,1,3
Hot,2,2
Mild,2,4


In [16]:
p_cool_yes = 3/10
p_hot_yes = 2/10
p_mild_yes = 4/10

p_cool_no = 1/5
p_hot_no = 2/5
p_mild_no = 2/5

In [17]:
#humidity
pd.crosstab(data['humidity'],data['play'])

play,No,Yes
humidity,,
High,4,3
Normal,1,6


In [18]:
p_high_yes = 3/9
p_low_yes = 6/9

p_high_no = 4/5
p_low_no = 1/5

In [19]:
#wind
pd.crosstab(data['wind'],data['play'])

play,No,Yes
wind,,
Strong,3,3
Weak,2,6


In [20]:
p_strong_yes = 3/9
p_weak_yes = 6/9

p_strong_no = 3/5
p_weak_no = 2/5

In [24]:
p_yes_final = pyes * p_sun_yes * p_cool_yes * p_low_yes * p_weak_yes
p_no_final = pno * p_sun_no * p_cool_no * p_low_no * p_weak_no

In [25]:
if(p_yes_final > p_no_final):
    print('Play')
else:
    print('Not Play')
    
print('probablity of playing',p_yes_final)
print('probablity of not playing',p_no_final)

Play
probablity of playing 0.019047619047619042
probablity of not playing 0.003428571428571429


# using Sklearn

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report

In [31]:
data = pd.read_csv('data/Social_Network_Ads.csv')
data = data.drop('User ID',axis=1)

In [32]:
data.head()

,Gender,Age,EstimatedSalary,Purchased
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0


In [33]:
for i in range(data.shape[0]):
    if data['Gender'][i] == 'Male':
        data['Gender'][i] = 1
    else :
        data['Gender'][i] = 0

C:\Users\91700\AppData\Local\Temp/ipykernel_3324/3713499745.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Gender'][i] = 1
C:\Users\91700\AppData\Local\Temp/ipykernel_3324/3713499745.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Gender'][i] = 0


In [34]:
data.head()

,Gender,Age,EstimatedSalary,Purchased
0,1,19,19000,0
1,1,35,20000,0
2,0,26,43000,0
3,0,27,57000,0
4,1,19,76000,0


In [35]:
X = data.drop('Purchased', axis=1)
y = data['Purchased']

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [39]:
lr = LogisticRegression()
dt = DecisionTreeClassifier()
knn = KNeighborsClassifier()
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [50]:
def evaluation(model,X,y,X_t,y_t):
    model.fit(X,y)
    y_pred = model.predict(X_t)
    print('Model Name ::::: ',model)
    print('---------Classification Report-----------')
    print(classification_report(y_t,y_pred))

def cross_validate(model,X,y):
    cv_score = cross_val_score(model,X,y,cv=10,scoring='accuracy')
    print('Model Name ::::: ',model)
    print('----------Cross Validation Score :::::::::   ',cv_score.mean()*100)

In [51]:
evaluation(gnb,X_train,y_train,X_test,y_test)

Model Name :::::  GaussianNB()
---------Classification Report-----------
              precision    recall  f1-score   support

           0       0.82      0.96      0.88        48
           1       0.92      0.69      0.79        32

    accuracy                           0.85        80
   macro avg       0.87      0.82      0.84        80
weighted avg       0.86      0.85      0.85        80



In [52]:
cross_validate(gnb,X,y)

Model Name :::::  GaussianNB()
----------Cross Validation Score :::::::::    87.5
